# VibeCurator - Stage 1.5: Metadata-based Re-ranking

이 노트북은 **Stage 1 (item2vec)**의 추천 결과에 **메타데이터(곡명, 아티스트, 장르)**를 결합하여 추천 품질을 보정(Re-ranking)하는 과정을 다룹니다.

### 주요 목표
1. **메타데이터 연동:** `song_meta.json`을 로드하여 추천된 곡의 상세 정보를 확인합니다.
2. **Re-ranking 적용:**
   - **Artist Penalty:** 동일 아티스트의 곡이 너무 많이 추천되는 것을 방지합니다.
   - **Genre Railguard:** 시드 곡과 너무 이질적인 장르가 섞이는 것을 방지합니다.
3. **품질 비교:** 순수 CF 결과와 Re-ranking 적용 후의 결과를 비교합니다.

In [ ]:
# [Cell 1] 공통 import 및 경로 설정

import json
import os
import logging
from pathlib import Path
from typing import List, Dict, Optional, Union

import numpy as np
import pandas as pd
from gensim.models import Word2Vec

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Pandas 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

# ==========================================
# 경로 설정 (recommend_model 폴더 기준)
# ==========================================
# 데이터 루트 디렉토리
DATA_ROOT = Path("../melon-dataset-excepttar")  # recommend_model 폴더 기준

# 메타데이터 파일 경로
META_PATH = DATA_ROOT / "song_meta.json"

# 모델 파일 경로 (Stage 1에서 저장한 모델)
MODEL_DIR = Path("../models")  # recommend_model 폴더 기준
MODEL_PATH = Path("../models/v2_item2vec.model")

logger.info(f"✅ 경로 설정 완료:")
logger.info(f"   - DATA_ROOT: {DATA_ROOT}")
logger.info(f"   - META_PATH: {META_PATH}")
logger.info(f"   - MODEL_PATH: {MODEL_PATH}")

✅ 경로 설정 완료:
   - DATA_ROOT: melon-dataset-excepttar
   - META_PATH: melon-dataset-excepttar\song_meta.json
   - MODEL_PATH: models\v2_item2vec.model


In [19]:
# [Cell 2] song_meta 로드 및 메타데이터 헬퍼

try:
    logger.info("📂 song_meta.json 로드 중...")
    # song_meta.json 로드
    with open(META_PATH, "r", encoding="utf-8") as f:
        song_meta_list = json.load(f)
    
    # DataFrame 변환
    song_meta_df = pd.DataFrame(song_meta_list)
    
    # 컬럼 전처리
    # 1. ID 컬럼 통일 (id -> song_id)
    if 'id' in song_meta_df.columns:
        song_meta_df.rename(columns={'id': 'song_id'}, inplace=True)
    
    # 2. artist_str 생성
    song_meta_df['artist_str'] = song_meta_df['artist_name_basket'].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else str(x)
    )

    # 3. artist_key 생성 (NEW: artist_id_basket 기반)
    def get_artist_key(id_basket):
        if isinstance(id_basket, list) and len(id_basket) > 0:
            return str(id_basket[0])
        return "UNKNOWN"

    if 'artist_id_basket' in song_meta_df.columns:
        song_meta_df['artist_key'] = song_meta_df['artist_id_basket'].apply(get_artist_key)
    else:
        song_meta_df['artist_key'] = "UNKNOWN"
    
    # 4. main_genre 추출 (첫 번째 장르 코드 사용)
    song_meta_df['main_genre'] = song_meta_df['song_gn_gnr_basket'].apply(
        lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "UNK"
    )
    
    # 5. issue_year 추출
    def parse_year(date_str):
        try:
            return int(str(date_str)[:4])
        except (ValueError, TypeError):
            return np.nan
            
    song_meta_df['issue_year'] = song_meta_df['issue_date'].apply(parse_year)
    
    # 6. 인덱스 설정
    song_meta_df.set_index('song_id', inplace=True)
    
    # 필요한 컬럼만 유지 (메모리 절약)
    cols_to_keep = ['song_name', 'artist_str', 'main_genre', 'issue_year', 'artist_key']
    song_meta_df = song_meta_df[cols_to_keep]
    
    logger.info(f"✅ 메타데이터 로드 완료: {len(song_meta_df):,}곡")
    
    # lookup 함수 정의
    def lookup_song_meta(song_id: Union[int, str]) -> Dict:
        try:
            sid = int(song_id)
            if sid in song_meta_df.index:
                return song_meta_df.loc[sid].to_dict()
        except ValueError:
            pass
            
        return {
            "song_name": None,
            "artist_str": None,
            "main_genre": None,
            "issue_year": None,
            "artist_key": None
        }

    # 데이터 확인
    display(song_meta_df.head(10))

except Exception as e:
    logger.error(f"❌ 메타데이터 로드 실패: {e}")
    raise

📂 song_meta.json 로드 중...
✅ 메타데이터 로드 완료: 707,989곡


,song_name,artist_str,main_genre,issue_year,artist_key
song_id,,,,,
0,Feelings,Various Artists,GN0900,2014,2727
1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",Murray Perahia,GN1600,2008,29966
2,Solsbury Hill (Remastered 2002),Peter Gabriel,GN0900,2018,3361
3,Feeling Right (Everything Is Nice) (Feat. Popc...,Matoma,GN1100,2015,838543
4,그남자 그여자,Jude Law,GN1800,2011,560160
5,Para Los Enamorados,"Bye, Lupita",GN1700,2010,475840
6,Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,"Herbert Blomstedt, San Francisco Symphony",GN1600,2017,162097
7,Superman March (From &#34;Superman&#34; / Live...,"Los Angeles Philharmonic Orchestra, Gustavo Du...",GN1600,2019,105210
8,Lovers’ Leap (Feat. Qypthone),애플스,GN0300,2007,224583


In [18]:
song_meta_df['artist_str'].head(10)

song_id
0                                      Various Artists
1                                       Murray Perahia
2                                        Peter Gabriel
3                                               Matoma
4                                             Jude Law
5                                          Bye, Lupita
6            Herbert Blomstedt, San Francisco Symphony
7    Los Angeles Philharmonic Orchestra, Gustavo Du...
8                                                  애플스
9                                                   진호
Name: artist_str, dtype: object

In [3]:
# [Cell 3] Stage1 item2vec 모델 로드

try:
    logger.info(f"🔄 모델 로드 중: {MODEL_PATH}")
    
    if not MODEL_PATH.exists():
        raise FileNotFoundError(f"모델 파일이 없습니다: {MODEL_PATH}")
        
    # 모델 로드
    item2vec_model = Word2Vec.load(str(MODEL_PATH))
    
    # 모델 정보 출력
    vocab_size = len(item2vec_model.wv)
    vector_dim = item2vec_model.wv.vector_size
    
    logger.info(f"✅ Loaded item2vec: vocab_size={vocab_size:,}, dim={vector_dim}")

except Exception as e:
    logger.error(f"❌ 모델 로드 실패: {e}")
    raise

🔄 모델 로드 중: models\v2_item2vec.model
loading Word2Vec object from models\v2_item2vec.model
loading wv recursively from models\v2_item2vec.model.wv.* with mmap=None
loading vectors from models\v2_item2vec.model.wv.vectors.npy with mmap=None
loading syn1neg from models\v2_item2vec.model.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': 'models\\v2_item2vec.model', 'datetime': '2025-11-28T14:57:12.474687', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
✅ Loaded item2vec: vocab_size=316,109, dim=128


In [4]:
# [Cell 4] 순수 CF 추천 함수 recommend_cf_raw (UPDATED: artist_key 포함)

def recommend_cf_raw(model, seed_song_id: Union[int, str], topn: int = 200) -> pd.DataFrame:
    """
    item2vec 모델을 사용하여 유사 곡을 추천하고 메타데이터를 결합합니다.
    """
    # 모델 키 타입에 맞게 변환 (str)
    seed_key = str(seed_song_id)
    
    if seed_key not in model.wv:
        logger.warning(f"⚠️ 곡 ID '{seed_key}'가 모델에 없습니다.")
        return pd.DataFrame()
        
    # 유사 곡 조회
    # seed 자기 자신 포함될 수 있으므로 topn + 1개 조회 후 필터링
    similar_items = model.wv.most_similar(seed_key, topn=topn+1)
    
    results = []
    for sid_str, score in similar_items:
        # 자기 자신 제외
        if sid_str == seed_key:
            continue
            
        sid = int(sid_str)
        meta = lookup_song_meta(sid)
        
        # UPDATED: artist_key 추가
        results.append({
            'song_id': sid,
            'score_cf': float(score),
            'song_name': meta.get('song_name'),
            'artist_str': meta.get('artist_str'),
            'main_genre': meta.get('main_genre'),
            'issue_year': meta.get('issue_year'),
            'artist_key': meta.get('artist_key', 'UNKNOWN')
        })
        
    # DataFrame 생성
    df = pd.DataFrame(results)
    
    # topn개만 자르기 (자기 자신 제외했으므로)
    df = df.head(topn)
    
    return df

In [5]:
# [Cell 5] 동일 아티스트 편중 완화 함수 (artist_key 기반, DEBUG 추가)

def apply_same_artist_penalty(df: pd.DataFrame, 
                            max_per_artist_soft: int = 3, 
                            penalty_per_extra: float = 0.05) -> pd.DataFrame:
    """
    동일 아티스트(artist_key) 곡이 너무 많이 나오면 소프트 페널티를 부여합니다.
    """
    if df.empty:
        return df
        
    # 원본 보존을 위해 복사
    df_new = df.copy()
    
    # artist_key가 결측인 경우 UNKNOWN 처리
    if 'artist_key' not in df_new.columns:
        df_new['artist_key'] = "UNKNOWN"
    
    df_new['artist_key'] = df_new['artist_key'].fillna("UNKNOWN")
    
    # score_cf 기준 정렬 (이미 되어있겠지만 확실히)
    df_new.sort_values('score_cf', ascending=False, inplace=True)

    # 그룹별 누적 카운트 (0부터 시작)
    df_new['artist_order_soft'] = df_new.groupby('artist_key').cumcount()
    
    # 페널티 계산
    def calculate_penalty(order):
        if order < max_per_artist_soft:
            return 0.0
        else:
            extra_count = order - max_per_artist_soft + 1
            return extra_count * penalty_per_extra
            
    df_new['artist_penalty_soft'] = df_new['artist_order_soft'].apply(calculate_penalty)
    
    # 점수 반영
    df_new['score_after_artist'] = df_new['score_cf'] - df_new['artist_penalty_soft']
    
    # 디버그용 (필요할 때만 주석 해제해서 사용)
    # logger.info("[DEBUG] apply_same_artist_penalty artist_key value_counts (top 5):")
    # logger.info(df_new['artist_key'].value_counts().head())
    
    return df_new

In [6]:
# [Cell 6] 최종 TopK에서 동일 아티스트 하드컷 함수 (UPDATED: 방어 코드 추가)

def apply_final_artist_hardcut(df: pd.DataFrame, 
                             topk_final: int = 30, 
                             max_per_artist_final: int = 2) -> pd.DataFrame:
    """
    최종 결과에서 아티스트당 최대 곡 수를 제한합니다 (Hardcut).
    """
    if df.empty:
        return df
    
    # UPDATED: artist_key 방어 코드
    if 'artist_key' not in df.columns:
        df = df.copy()
        df['artist_key'] = "UNKNOWN"
        
    # score_final 기준 정렬
    if 'score_final' in df.columns:
        df_sorted = df.sort_values('score_final', ascending=False)
    else:
        df_sorted = df
    
    # UPDATED: artist_key 결측값 처리
    df_sorted = df_sorted.copy()
    df_sorted['artist_key'] = df_sorted['artist_key'].fillna("UNKNOWN")

    selected_rows = []
    artist_counts = {}
    
    for idx, row in df_sorted.iterrows():
        # Top K 다 채웠으면 중단
        if len(selected_rows) >= topk_final:
            break
            
        a_key = row.get('artist_key', "UNKNOWN")
        
        current_count = artist_counts.get(a_key, 0)
        
        if current_count < max_per_artist_final:
            selected_rows.append(row)
            artist_counts[a_key] = current_count + 1
            
    # DataFrame 생성
    df_final = pd.DataFrame(selected_rows)
    
    # 인덱스 리셋
    if not df_final.empty:
        df_final.reset_index(drop=True, inplace=True)
    
    return df_final

In [7]:
# [Cell 7] 장르 레일가드 함수 (UPDATED: 페널티 대폭 완화 + 로직 개선)

def apply_genre_railguard(df: pd.DataFrame, 
                        seed_main_genre: str, 
                        offrail_penalty_general: float = 0.008,  # UPDATED: 0.02 -> 0.008
                        offrail_penalty_special: float = 0.03) -> pd.DataFrame:  # UPDATED: 0.05 -> 0.03
    """
    시드 곡과 장르가 다르면 페널티를 부여합니다.
    특수 장르(트로트, CCM 등)는 더 강한 페널티를 적용합니다.
    """
    if df.empty:
        return df
        
    df_new = df.copy()
    
    # 헬퍼: 장르 그룹핑
    def get_genre_group(code):
        if not isinstance(code, str) or not code:
            return "UNK"
        
        # 특수 장르 하드코딩 예시
        if code.startswith("GN07") or code.startswith("GN11"): # 트로트/성인가요 등 (예시)
            return "TROT"
        if code == "GN1900" or code.startswith("GN19"): # CCM/종교
            return "CCM"
        if code == "GN2200" or code.startswith("GN22"): # 어린이/태교
            return "KIDS"
        if code == "GN2400" or code.startswith("GN24"): # 국악
            return "GUGAK"
            
        # 일반 장르는 앞 4자리 (GN0900 -> GN09)
        if len(code) >= 4:
            return code[:4]
            
        return "UNK"

    seed_genre_group = get_genre_group(seed_main_genre)
    
    # seed 장르가 UNK이면 페널티 적용하지 않음
    if seed_genre_group == "UNK":
        df_new['genre_penalty'] = 0.0
        df_new['score_after_genre'] = df_new['score_after_artist']
        return df_new

    # UPDATED: 로직 개선
    def calculate_genre_penalty(genre):
        cand_group = get_genre_group(genre)
        
        # 완전 동일 그룹이면 페널티 없음
        if cand_group == seed_genre_group:
            return 0.0
            
        special_groups = ["TROT", "CCM", "KIDS", "GUGAK"]
        
        # 둘 다 특수 그룹인데 서로 다르면 강한 페널티
        if seed_genre_group in special_groups and cand_group in special_groups:
            return offrail_penalty_special
        
        # 한쪽만 특수 그룹이고 다른 한쪽은 일반 그룹인 경우: 일반 페널티보다 조금 더 강하게
        if (seed_genre_group in special_groups) != (cand_group in special_groups):
            return offrail_penalty_general * 1.5
        
        # 일반 그룹 vs 일반 그룹, 서로 다른 경우: 아주 약한 페널티
        return offrail_penalty_general
        
    df_new['genre_penalty'] = df_new['main_genre'].apply(calculate_genre_penalty)
    
    # 점수 반영
    df_new['score_after_genre'] = df_new['score_after_artist'] - df_new['genre_penalty']
    
    return df_new

In [8]:
# [Cell 8] Stage1.5 최종 추천 함수 (UPDATED: 새 페널티 값 적용)

def recommend_cf_with_rerank(model, 
                           seed_song_id: Union[int, str], 
                           topn_cf: int = 200, 
                           topk_final: int = 30,
                           max_per_artist_soft: int = 3,
                           max_per_artist_final: int = 2) -> pd.DataFrame:
    """
    CF 후보 추출 -> 아티스트 페널티(Soft) -> 장르 레일가드 -> 최종 정렬 -> 아티스트 하드컷
    """
    # 1. CF 후보 추출
    df = recommend_cf_raw(model, seed_song_id, topn=topn_cf)
    
    if df.empty:
        return df
        
    # 2. Seed 정보 조회
    seed_meta = lookup_song_meta(seed_song_id)
    seed_main_genre = seed_meta.get('main_genre', "UNK")
    
    # 3. Artist Penalty (Soft) 적용
    df = apply_same_artist_penalty(df, 
                                   max_per_artist_soft=max_per_artist_soft,
                                   penalty_per_extra=0.05)
    
    # 4. Genre Railguard 적용 (UPDATED: 새 페널티 값)
    df = apply_genre_railguard(df, 
                              seed_main_genre=seed_main_genre,
                              offrail_penalty_general=0.008,
                              offrail_penalty_special=0.03)
    
    # 5. 최종 점수 계산 및 정렬
    df['score_final'] = df['score_after_genre']
    # score_final 기준 내림차순
    df.sort_values(by='score_final', ascending=False, inplace=True)
    
    # 6. Artist Hardcut 적용 (여기서 Top K가 결정됨)
    df_final = apply_final_artist_hardcut(df, 
                                          topk_final=topk_final, 
                                          max_per_artist_final=max_per_artist_final)
    
    # 컬럼 정리
    cols = ['song_id', 'song_name', 'artist_str', 'main_genre', 'issue_year', 
            'score_cf', 'score_after_artist', 'score_after_genre', 'score_final']
    # 존재하는 컬럼만 선택
    cols = [c for c in cols if c in df_final.columns]
    
    return df_final[cols]

In [9]:
# [Cell 9] Stage1 vs Stage1.5 다중 시드 평가

def evaluate_stage1_5(model, seed_ids: List[int], topk: int = 30):
    print(f"📊 다중 시드 평가 (Seeds: {len(seed_ids)}개)")
    print("="*60)
    
    results = []
    
    for seed_id in seed_ids:
        # 1. Raw CF
        df_raw = recommend_cf_raw(model, seed_id, topn=topk)
        
        # 2. Rerank
        df_rerank = recommend_cf_with_rerank(model, seed_id, topn_cf=200, topk_final=topk)
        
        # 메트릭 계산
        res = {
            'seed_id': seed_id,
            'raw_artists': df_raw['artist_str'].nunique() if not df_raw.empty else 0,
            'raw_genres': df_raw['main_genre'].nunique() if not df_raw.empty else 0,
            'rerank_artists': df_rerank['artist_str'].nunique() if not df_rerank.empty else 0,
            'rerank_genres': df_rerank['main_genre'].nunique() if not df_rerank.empty else 0
        }
        results.append(res)
        
    # DataFrame으로 변환
    eval_df = pd.DataFrame(results)
    
    # 요약 통계
    print(f"\n[평균 다양성 지표 (Top {topk})]")
    print(f" - Unique Artists: Raw {eval_df['raw_artists'].mean():.1f} -> Rerank {eval_df['rerank_artists'].mean():.1f}")
    print(f" - Unique Genres : Raw {eval_df['raw_genres'].mean():.1f} -> Rerank {eval_df['rerank_genres'].mean():.1f}")
    
    return eval_df

# 테스트 실행 (랜덤 샘플링 또는 지정)
# 예시로 song_meta_df에서 랜덤하게 5개만 뽑아서 테스트
if 'song_meta_df' in locals():
    sample_seeds = np.random.choice(song_meta_df.index, 5, replace=False)
    evaluate_stage1_5(item2vec_model, sample_seeds)
else:
    print("song_meta_df가 로드되지 않아 테스트를 건너뜁니다.")

⚠️ 곡 ID '220454'가 모델에 없습니다.
⚠️ 곡 ID '220454'가 모델에 없습니다.


⚠️ 곡 ID '435170'가 모델에 없습니다.
⚠️ 곡 ID '435170'가 모델에 없습니다.
⚠️ 곡 ID '284489'가 모델에 없습니다.
⚠️ 곡 ID '284489'가 모델에 없습니다.
⚠️ 곡 ID '248402'가 모델에 없습니다.
⚠️ 곡 ID '248402'가 모델에 없습니다.


📊 다중 시드 평가 (Seeds: 5개)

[평균 다양성 지표 (Top 30)]
 - Unique Artists: Raw 2.6 -> Rerank 4.6
 - Unique Genres : Raw 0.4 -> Rerank 0.8


In [ ]:
# [Cell 10] Stage1 vs Stage1.5 시각적 비교 (고정 시드 3개, Top 10)

from gensim.models import Word2Vec
from IPython.display import display, HTML
import random
import numpy as np

# 1) 모델 로드 (recommend_model 폴더 기준)
MODEL_PATH = Path("../models/v2_item2vec.model")
current_model = Word2Vec.load(str(MODEL_PATH))
logger.info(f"✅ Loaded item2vec model from {MODEL_PATH}")

# 2) 모델 vocab과 메타데이터 교집합 구하기
model_vocab_ids = [int(k) for k in current_model.wv.key_to_index.keys()]
candidate_ids = list(set(song_meta_df.index) & set(model_vocab_ids))

if len(candidate_ids) < 3:
    raise ValueError("모델+메타 교집합에서 시드로 사용할 수 있는 곡이 3개 미만입니다.")

# 3) 시드 곡 3개 무작위 추출 (재현성 확보)
random.seed(42)
seed_ids = random.sample(candidate_ids, 3)

logger.info(f"🎯 선택된 시드 곡 ID: {seed_ids}")

# 4) 각 시드에 대해 Before/After Top 10 비교
for seed_id in seed_ids:
    # Seed 정보 조회
    seed_meta = lookup_song_meta(seed_id)
    song_name = seed_meta.get("song_name") or "UNKNOWN"
    artist_str = seed_meta.get("artist_str") or "UNKNOWN"
    main_genre = seed_meta.get("main_genre") or "UNK"
    
    # 헤더 출력
    header_html = f"""
    <h3 style="color: #2c3e50;">🎯 Seed: {song_name} - {artist_str}</h3>
    <p style="margin: 5px 0;"><b>ID:</b> {seed_id} &nbsp; | &nbsp; <b>Genre:</b> {main_genre}</p>
    """
    display(HTML(header_html))
    
    # Raw CF 추천
    raw_df = recommend_cf_raw(current_model, seed_id, topn=30)
    if raw_df.empty:
        display(HTML("<p style='color:red'>⚠️ 이 시드는 모델에서 추천을 생성하지 못했습니다 (raw).</p>"))
        continue
    
    # Stage1.5 Rerank 추천
    rerank_df = recommend_cf_with_rerank(current_model, seed_id, topn_cf=200, topk_final=30)
    if rerank_df.empty:
        display(HTML("<p style='color:red'>⚠️ 이 시드는 Stage1.5 rerank 결과가 비어 있습니다.</p>"))
        continue
    
    # Top 10만 추출
    raw_top10 = raw_df.head(10)
    rerank_top10 = rerank_df.head(10)
    
    # 컬럼 선택 (존재하는 것만)
    raw_cols = ['song_name', 'artist_str', 'main_genre', 'score_cf']
    raw_cols = [c for c in raw_cols if c in raw_top10.columns]
    
    rerank_cols = ['song_name', 'artist_str', 'main_genre', 'score_cf',
                   'score_after_artist', 'score_after_genre', 'score_final',
                   'artist_penalty_soft', 'genre_penalty']
    rerank_cols = [c for c in rerank_cols if c in rerank_top10.columns]
    
    # Before 출력
    display(HTML("<h4 style='color: #e74c3c;'>[Before] Pure CF (Top 10)</h4>"))
    display(raw_top10[raw_cols])
    
    # After 출력
    display(HTML("<h4 style='color: #27ae60;'>[After] Stage1.5 Rerank (Top 10)</h4>"))
    display(rerank_top10[rerank_cols])
    
    # 구분선
    display(HTML("<hr style='border: 2px solid #bdc3c7; margin: 30px 0;'>"))

logger.info("✅ Stage1 vs Stage1.5 비교 완료")

loading Word2Vec object from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model
loading wv recursively from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.wv.* with mmap=None
loading vectors from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.wv.vectors.npy with mmap=None
loading syn1neg from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': 'C:\\Users\\ASUS\\music_recommend\\work\\models\\v2_item2vec.model', 'datetime': '2025-11-28T14:57:15.009177', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
✅ Loaded item2vec model from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model
🎯 선택된 시드 곡 ID: [130956, 29452, 322951]


,song_name,artist_str,main_genre,score_cf
0,I Never Woke Up In Handcuffs Before,Various Artists,GN1500,0.990164
1,Who Wants To Live Forever,Amaury Vassili,GN1600,0.986615
2,Palm Tree Escape (From &#34;Pirates of the Car...,Hans Zimmer,GN1500,0.985482
3,You Raise Me Up 유 레이즈 미 업 (해금연주) (파도소리),Various Artists,GN1800,0.985051
4,Justice,모그,GN1500,0.984800
5,Magic Moments,Perry Como,GN0900,0.984082
6,The Dashing Young Falconer,John Debney,GN1500,0.982199
7,Namaewo Nakushita Megami (이름을 잃은 여신),Izutsu Akio,GN1500,0.981844
8,우리에게 내일은 없다,Various Artists,GN1500,0.981241
9,The Prayer,Jonathan & Charlotte,GN1600,0.980800


,song_name,artist_str,main_genre,score_cf,score_after_artist,score_after_genre,score_final
0,I Never Woke Up In Handcuffs Before,Various Artists,GN1500,0.990164,0.990164,0.990164,0.990164
1,Palm Tree Escape (From &#34;Pirates of the Car...,Hans Zimmer,GN1500,0.985482,0.985482,0.985482,0.985482
2,Justice,모그,GN1500,0.984800,0.984800,0.984800,0.984800
3,The Dashing Young Falconer,John Debney,GN1500,0.982199,0.982199,0.982199,0.982199
4,Namaewo Nakushita Megami (이름을 잃은 여신),Izutsu Akio,GN1500,0.981844,0.981844,0.981844,0.981844
5,우리에게 내일은 없다,Various Artists,GN1500,0.981241,0.981241,0.981241,0.981241
6,Ranchino Zyoou / ランチの女王 (후지TV 드라마 '런치의 여왕') (F...,Yoshimata Ryo,GN1500,0.980513,0.980513,0.980513,0.980513
7,흘러가다,연리목,GN1500,0.979992,0.979992,0.979992,0.979992
8,Giochi Probit,The Film Studio Orchestra,GN1500,0.979802,0.979802,0.979802,0.979802
9,Think Of Me (Live At The Royal Albert Hall/2011),Andrew Lloyd Webber,GN1500,0.979471,0.979471,0.979471,0.979471


,song_name,artist_str,main_genre,score_cf
0,Monicas Vals,Monica Zetterlund,GN1700,0.995879
1,Deep In A Dream (2008 Digital Remaster) (Rudy ...,Sonny Clark,GN1700,0.995652
2,Sol`s Song,Trygve Seim,GN1700,0.994749
3,Winter Weather,Joe Williams,GN1700,0.994746
4,Wave,Singers Unlimited,GN1700,0.994624
5,The Monterey Suite : I. Bring It On,Dave Holland,GN1700,0.994333
6,It&#39;s Easy To Remember (Album Version),Seth MacFarlane,GN1700,0.994327
7,I`m A Fool To Want You,Dominick Farinacci Quartet,GN1700,0.994237
8,Smoke Gets in Your Eyes (Album Version),Arturo Sandoval,GN1700,0.994236
9,Trav`lin` All Alone,Billie Holiday,GN1700,0.994221


,song_name,artist_str,main_genre,score_cf,score_after_artist,score_after_genre,score_final
0,Monicas Vals,Monica Zetterlund,GN1700,0.995879,0.995879,0.995879,0.995879
1,Deep In A Dream (2008 Digital Remaster) (Rudy ...,Sonny Clark,GN1700,0.995652,0.995652,0.995652,0.995652
2,Sol`s Song,Trygve Seim,GN1700,0.994749,0.994749,0.994749,0.994749
3,Winter Weather,Joe Williams,GN1700,0.994746,0.994746,0.994746,0.994746
4,Wave,Singers Unlimited,GN1700,0.994624,0.994624,0.994624,0.994624
5,The Monterey Suite : I. Bring It On,Dave Holland,GN1700,0.994333,0.994333,0.994333,0.994333
6,It&#39;s Easy To Remember (Album Version),Seth MacFarlane,GN1700,0.994327,0.994327,0.994327,0.994327
7,I`m A Fool To Want You,Dominick Farinacci Quartet,GN1700,0.994237,0.994237,0.994237,0.994237
8,Smoke Gets in Your Eyes (Album Version),Arturo Sandoval,GN1700,0.994236,0.994236,0.994236,0.994236
9,Trav`lin` All Alone,Billie Holiday,GN1700,0.994221,0.994221,0.994221,0.994221


,song_name,artist_str,main_genre,score_cf
0,Close To You,Fried Pride,GN1700,0.937861
1,그저 그렇고 그런 기억,푸디토리움,GN1700,0.934843
2,Take On Me (Aha),Be The Voice,GN1900,0.933539
3,Viva La Vida (Coldplay),Freedom Orchestra,GN1700,0.933531
4,Too Shy To Say (Dedicated To Stevie Wonder),정엽,GN0400,0.932945
5,One More Time On The Merry-Go-Round,Wouter Hamel,GN1700,0.930189
6,Sans rancune (썽 헝뀐느) (그저 그렇고 그런 기억 French Ver....,푸디토리움,GN1700,0.928137
7,Love Box (Feat. 타루),Maximilian Hecker,GN0600,0.926989
8,In Seoul,하이사이드,GN0400,0.926705
9,Ran Into Bookstore,Naomi & Goro,GN1900,0.926632


,song_name,artist_str,main_genre,score_cf,score_after_artist,score_after_genre,score_final
0,Close To You,Fried Pride,GN1700,0.937861,0.937861,0.937861,0.937861
1,그저 그렇고 그런 기억,푸디토리움,GN1700,0.934843,0.934843,0.934843,0.934843
2,Viva La Vida (Coldplay),Freedom Orchestra,GN1700,0.933531,0.933531,0.933531,0.933531
3,One More Time On The Merry-Go-Round,Wouter Hamel,GN1700,0.930189,0.930189,0.930189,0.930189
4,Sans rancune (썽 헝뀐느) (그저 그렇고 그런 기억 French Ver....,푸디토리움,GN1700,0.928137,0.928137,0.928137,0.928137
5,Too Shy To Say (Dedicated To Stevie Wonder),정엽,GN0400,0.932945,0.932945,0.924945,0.924945
6,Scandalizing Me,윈터플레이,GN1700,0.924489,0.924489,0.924489,0.924489
7,Have I Told You Lately That I Love You,Inger Marie,GN1700,0.923508,0.923508,0.923508,0.923508
8,I Only Want To Be With You,Inger Marie,GN1700,0.922560,0.922560,0.922560,0.922560
9,Dream Street (꿈에서 본 거리),윈터플레이,GN1700,0.921894,0.921894,0.921894,0.921894


✅ Stage1 vs Stage1.5 비교 완료


In [ ]:
# [Cell 11] 특정 아티스트 시드 테스트 (ZUTOMAYO, 아이유, 잔나비)

from gensim.models import Word2Vec
from IPython.display import display, HTML
import random
import numpy as np

# 1) 모델 로드 (recommend_model 폴더 기준)
MODEL_PATH = Path("../models/v2_item2vec.model")
current_model = Word2Vec.load(str(MODEL_PATH))
logger.info(f"✅ Loaded item2vec model from {MODEL_PATH}")

# 2) 특정 아티스트의 곡 찾기
target_artists = ["ZUTOMAYO", "아이유", "잔나비"]
seed_ids = []

model_vocab_ids = set([int(k) for k in current_model.wv.key_to_index.keys()])

for artist_name in target_artists:
    # artist_str에 아티스트 이름이 포함된 곡 찾기
    artist_songs = song_meta_df[song_meta_df['artist_str'].str.contains(artist_name, case=False, na=False)]
    
    # 모델에 존재하는 곡만 필터링
    valid_songs = artist_songs[artist_songs.index.isin(model_vocab_ids)]
    
    if len(valid_songs) == 0:
        logger.warning(f"⚠️ '{artist_name}' 아티스트의 곡이 모델에 없습니다.")
        continue
    
    # 첫 번째 곡 선택 (또는 랜덤)
    selected_song_id = valid_songs.index[0]
    seed_ids.append(selected_song_id)
    
    song_info = valid_songs.loc[selected_song_id]
    logger.info(f"✅ {artist_name}: '{song_info['song_name']}' (ID: {selected_song_id})")

if len(seed_ids) < 3:
    logger.error(f"❌ 선택된 시드 곡이 {len(seed_ids)}개뿐입니다. 일부 아티스트의 곡이 모델에 없을 수 있습니다.")

logger.info(f"🎯 선택된 시드 곡 ID: {seed_ids}")

# 3) 각 시드에 대해 Before/After Top 10 비교
for seed_id in seed_ids:
    # Seed 정보 조회
    seed_meta = lookup_song_meta(seed_id)
    song_name = seed_meta.get("song_name") or "UNKNOWN"
    artist_str = seed_meta.get("artist_str") or "UNKNOWN"
    main_genre = seed_meta.get("main_genre") or "UNK"
    
    # 헤더 출력
    header_html = f"""
    <h3 style="color: #2c3e50;">🎯 Seed: {song_name} - {artist_str}</h3>
    <p style="margin: 5px 0;"><b>ID:</b> {seed_id} &nbsp; | &nbsp; <b>Genre:</b> {main_genre}</p>
    """
    display(HTML(header_html))
    
    # Raw CF 추천
    raw_df = recommend_cf_raw(current_model, seed_id, topn=30)
    if raw_df.empty:
        display(HTML("<p style='color:red'>⚠️ 이 시드는 모델에서 추천을 생성하지 못했습니다 (raw).</p>"))
        continue
    
    # Stage1.5 Rerank 추천
    rerank_df = recommend_cf_with_rerank(current_model, seed_id, topn_cf=200, topk_final=30)
    if rerank_df.empty:
        display(HTML("<p style='color:red'>⚠️ 이 시드는 Stage1.5 rerank 결과가 비어 있습니다.</p>"))
        continue
    
    # Top 10만 추출
    raw_top10 = raw_df.head(10)
    rerank_top10 = rerank_df.head(10)
    
    # 컬럼 선택 (존재하는 것만)
    raw_cols = ['song_name', 'artist_str', 'main_genre', 'score_cf']
    raw_cols = [c for c in raw_cols if c in raw_top10.columns]
    
    rerank_cols = ['song_name', 'artist_str', 'main_genre', 'score_cf',
                   'score_after_artist', 'score_after_genre', 'score_final',
                   'artist_penalty_soft', 'genre_penalty']
    rerank_cols = [c for c in rerank_cols if c in rerank_top10.columns]
    
    # Before 출력
    display(HTML("<h4 style='color: #e74c3c;'>[Before] Pure CF (Top 10)</h4>"))
    display(raw_top10[raw_cols])
    
    # After 출력
    display(HTML("<h4 style='color: #27ae60;'>[After] Stage1.5 Rerank (Top 10)</h4>"))
    display(rerank_top10[rerank_cols])
    
    # 구분선
    display(HTML("<hr style='border: 2px solid #bdc3c7; margin: 30px 0;'>"))

logger.info("✅ 특정 아티스트 시드 테스트 완료")

loading Word2Vec object from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model
loading wv recursively from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.wv.* with mmap=None
loading vectors from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.wv.vectors.npy with mmap=None
loading syn1neg from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': 'C:\\Users\\ASUS\\music_recommend\\work\\models\\v2_item2vec.model', 'datetime': '2025-11-28T14:57:17.835692', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
✅ Loaded item2vec model from C:\Users\ASUS\music_recommend\work\models\v2_item2vec.model
⚠️ 'ZUTOMAYO' 아티스트의 곡이 모델에 없습니다.
✅ 아이유: '을의 연애 (With 박주원)' (ID: 261)
✅ 잔나비: 'The Secret Of Hard Rock' (ID: 2010)
❌ 선택

,song_name,artist_str,main_genre,score_cf
0,아이야 나랑 걷자 (Feat. 최백호),아이유,GN0100,0.950036
1,누구나 비밀은 있다 (Feat. 가인 Of Brown Eyed Girls),아이유,GN0200,0.945398
2,"잔소리 (원곡가수 아이유, 임슬옹)","이도진, 김연준",GN0100,0.942396
3,Obliviate,아이유,GN0600,0.941526
4,기다려,아이유,GN0200,0.932342
5,입술 사이 (50cm),아이유,GN0600,0.931024
6,Bonus Track - 너랑 나 (FPM TECHNORCHESTRA MIX),아이유,GN0200,0.929952
7,Modern Times,아이유,GN0600,0.922756
8,Obliviate,아이유,GN0600,0.919040
9,Modern Times,아이유,GN0600,0.917591


,song_name,artist_str,main_genre,score_cf,score_after_artist,score_after_genre,score_final
0,아이야 나랑 걷자 (Feat. 최백호),아이유,GN0100,0.950036,0.950036,0.942036,0.942036
1,Obliviate,아이유,GN0600,0.941526,0.941526,0.941526,0.941526
2,"잔소리 (원곡가수 아이유, 임슬옹)","이도진, 김연준",GN0100,0.942396,0.942396,0.934396,0.934396
3,아이유(I & U),유아이,GN0200,0.913480,0.913480,0.905480,0.905480
4,Dreamer (Narr. 아이유),히스토리,GN0200,0.893645,0.893645,0.885645,0.885645
5,"어떤날, 너에게",임재범,GN0100,0.878850,0.878850,0.870850,0.870850
6,Mystery Survivor,브라운아이드걸스,GN2500,0.869363,0.869363,0.861363,0.861363
7,여러분,임재범,GN0100,0.862668,0.862668,0.854668,0.854668
8,그대와 함께,플랜비,GN0600,0.852419,0.852419,0.852419,0.852419
9,Good Day To Die,제이투엠,GN0600,0.851221,0.851221,0.851221,0.851221


,song_name,artist_str,main_genre,score_cf
0,꿈나라 별나라,잔나비,GN0500,0.951767
1,MONKEY HOTEL (Finale),잔나비,GN0500,0.942032
2,내 마음에 주단을 깔고,라이프 앤 타임,GN0500,0.937068
3,Wish,잔나비,GN0500,0.927244
4,Ticket to the Moon,솔루션스 (THE SOLUTIONS),GN0500,0.924456
5,HONG KONG,잔나비,GN0500,0.924085
6,Surprise!,잔나비,GN0500,0.916409
7,석류의 맛,쏜애플 (THORNAPPLE),GN0500,0.915785
8,세상만사,라이프 앤 타임,GN0500,0.913709
9,STEREOTYPES,라이엇 키즈(RIOT KIDZ),GN0500,0.910403


,song_name,artist_str,main_genre,score_cf,score_after_artist,score_after_genre,score_final
0,꿈나라 별나라,잔나비,GN0500,0.951767,0.951767,0.951767,0.951767
1,MONKEY HOTEL (Finale),잔나비,GN0500,0.942032,0.942032,0.942032,0.942032
2,내 마음에 주단을 깔고,라이프 앤 타임,GN0500,0.937068,0.937068,0.937068,0.937068
3,Ticket to the Moon,솔루션스 (THE SOLUTIONS),GN0500,0.924456,0.924456,0.924456,0.924456
4,석류의 맛,쏜애플 (THORNAPPLE),GN0500,0.915785,0.915785,0.915785,0.915785
5,세상만사,라이프 앤 타임,GN0500,0.913709,0.913709,0.913709,0.913709
6,STEREOTYPES,라이엇 키즈(RIOT KIDZ),GN0500,0.910403,0.910403,0.910403,0.910403
7,밤밤밤,Adios Audio,GN0500,0.907904,0.907904,0.907904,0.907904
8,Spaceship,문댄서즈 (MDSZ),GN0500,0.903542,0.903542,0.903542,0.903542
9,Party Maker,The Hans,GN0500,0.902563,0.902563,0.902563,0.902563


✅ 특정 아티스트 시드 테스트 완료


In [15]:
# JPOP Top 10 분석 (플레이리스트 빈도 기준)

import pandas as pd

# 1) 모델 vocab의 빈도 정보 추출
vocab_freq = {}
for word in item2vec_model.wv.index_to_key:
    song_id = int(word)
    count = item2vec_model.wv.get_vecattr(word, "count")
    vocab_freq[song_id] = count

# 2) DataFrame으로 변환
freq_df = pd.DataFrame(list(vocab_freq.items()), columns=['song_id', 'playlist_count'])
freq_df = freq_df.sort_values('playlist_count', ascending=False)

# 3) 메타데이터와 결합
freq_df = freq_df.merge(
    song_meta_df[['song_name', 'artist_str', 'main_genre']], 
    left_on='song_id', 
    right_index=True, 
    how='left'
)

# 4) JPOP 필터링 (GN1900)
jpop_top10 = freq_df[freq_df['main_genre'] == 'GN1900'].head(20)

# 5) 결과 출력
print("=" * 80)
print("🎌 학습 데이터 중 가장 많이 플레이리스트에 담긴 JPOP Top 10")
print("=" * 80)
print(f"\n{'순위':<5} {'곡명':<40} {'아티스트':<25} {'플레이리스트 수':>12}")
print("-" * 80)

for idx, (rank, row) in enumerate(jpop_top10.iterrows(), 1):
    song_name = row['song_name'][:38] + '..' if len(str(row['song_name'])) > 40 else row['song_name']
    artist = row['artist_str'][:23] + '..' if len(str(row['artist_str'])) > 25 else row['artist_str']
    count = row['playlist_count']
    
    print(f"{idx:<5} {song_name:<40} {artist:<25} {count:>12,}")

print("\n✅ 분석 완료!")

# 추가: JPOP 전체 통계
total_jpop = freq_df[freq_df['main_genre'] == 'GN1900'].shape[0]
total_songs = len(freq_df)
jpop_ratio = (total_jpop / total_songs) * 100

print(f"\n📊 JPOP 통계:")
print(f"  - 전체 학습 곡 수: {total_songs:,}곡")
print(f"  - JPOP 곡 수: {total_jpop:,}곡 ({jpop_ratio:.1f}%)")


🎌 학습 데이터 중 가장 많이 플레이리스트에 담긴 JPOP Top 10

순위    곡명                                       아티스트                          플레이리스트 수
--------------------------------------------------------------------------------
1     Flower Dance                             DJ Okawari                         501
2     Close To You                             Olivia Ong                         282
3     Consider My Love (Feat. Pismo)           Nishihara Kenichiro                250
4     Clean & Dirty                            Harvard                            206
5     Altogether Alone                         Be The Voice                       204
6     Dreaming (Feat. Nami Miyahara)           FreeTEMPO                          194
7     Marry Me                                 Ellegarden                         188
8     Romance For Journey                      Daishi Dance                       182
9     Sky High (Feat. Blanc.)                  FreeTEMPO                          182
10    Rain        

In [16]:
# 전체 장르 Top 10 분석 (플레이리스트 빈도 기준)

import pandas as pd

# 1) 모델 vocab의 빈도 정보 추출
vocab_freq = {}
for word in item2vec_model.wv.index_to_key:
    song_id = int(word)
    count = item2vec_model.wv.get_vecattr(word, "count")
    vocab_freq[song_id] = count

# 2) DataFrame으로 변환
freq_df = pd.DataFrame(list(vocab_freq.items()), columns=['song_id', 'playlist_count'])
freq_df = freq_df.sort_values('playlist_count', ascending=False)

# 3) 메타데이터와 결합
freq_df = freq_df.merge(
    song_meta_df[['song_name', 'artist_str', 'main_genre']], 
    left_on='song_id', 
    right_index=True, 
    how='left'
)

# 4) 전체 Top 10 추출
top10 = freq_df.head(10)

# 5) 결과 출력
print("=" * 85)
print("🔥 학습 데이터 중 가장 많이 플레이리스트에 담긴 곡 Top 10 (전체 장르)")
print("=" * 85)
print(f"\n{'순위':<5} {'곡명':<40} {'아티스트':<25} {'장르':<8} {'플레이리스트 수':>12}")
print("-" * 85)

for idx, (rank, row) in enumerate(top10.iterrows(), 1):
    song_name = row['song_name'][:38] + '..' if len(str(row['song_name'])) > 40 else row['song_name']
    artist = row['artist_str'][:23] + '..' if len(str(row['artist_str'])) > 25 else row['artist_str']
    genre = row['main_genre']
    count = row['playlist_count']
    
    print(f"{idx:<5} {song_name:<40} {artist:<25} {genre:<8} {count:>12,}")

print("\n✅ 분석 완료!")

# 추가: 장르별 분포
print(f"\n📊 Top 10 장르 분포:")
genre_dist = top10['main_genre'].value_counts()
for genre, count in genre_dist.items():
    print(f"  - {genre}: {count}곡")

🔥 학습 데이터 중 가장 많이 플레이리스트에 담긴 곡 Top 10 (전체 장르)

순위    곡명                                       아티스트                      장르           플레이리스트 수
-------------------------------------------------------------------------------------
1     밤편지                                      아이유                       GN0100          2,175
2     안아줘                                      정준일                       GN0500          2,121
3     비                                        폴킴                        GN0400          1,981
4     가끔 미치도록 네가 안고 싶어질 때가 있어                  가을방학                      GN0500          1,919
5     눈의 꽃                                     박효신                       GN1500          1,647
6     그대와 나, 설레임 (Feat. 소울맨)                   어쿠스틱 콜라보                  GN0500          1,633
7     어떤이별                                     임승부                       GN0100          1,599
8     야생화                                      박효신                       GN0100          1,598
9     비가 내렸어 